<h1 style="color:rgb(255, 98, 0);">1. Simple Chain</h1>

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template='Generate 3 interesting facts about {topic}',
    input_variables=['topic']
)

model = OllamaLLM(model="llama3.2")

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

print(result)

Here are three interesting facts about cricket:

1. **The Origin of Cricket**: The origins of cricket are still debated, but it is believed to have originated in England in the 16th century as a game played by the English public schools, such as Eton and Harrow. The modern version of the game emerged in the 18th century, with the first official rules being published in 1744.

2. **The Art of Spin Bowling**: Cricket has one of the most complex and fascinating forms of bowling - spin bowling. There are several types of spin bowling, including off-spin, leg-spin, and top-spin, each requiring a specific technique to execute. The art of spin bowling is considered one of the most difficult aspects of cricket, with only a handful of bowlers in the world able to master it.

3. **The World's Longest Cricket Match**: In 1939, England and South Africa played a test match that lasted for six days - a record that still stands today. The match was played at Lord's Cricket Ground in London, which hos

In [5]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +-----------+        
      | OllamaLLM |        
      +-----------+        
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


<h1 style="color:rgb(255, 98, 0);">2. Sequential Chain</h1>

In [6]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Generate Detailed Report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate 3 key points summary of the following text \n {text}'
)

model = OllamaLLM(model="llama3.2")

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic':'Is IPL game of Gambling?'})

In [7]:
print(result)

Here are 3 key points summarizing the text:

1. **The IPL has become linked to gambling**: The Indian Premier League (IPL) has seen an increase in online betting platforms, social media promotions, and fantasy sports, making it easier for fans to engage with their favorite teams and participate in betting markets.

2. **The rise of IPL-related gambling poses significant concerns**: Problem gambling, financial incentives, and social impact are major concerns associated with IPL-related gambling. Bookmakers offer attractive odds and incentives, tempting fans to invest more money than they can afford, while the lack of regulation has led to devastating effects on individuals and communities.

3. **Regulatory measures and public awareness campaigns are needed**: To mitigate the risks associated with IPL-related gambling, it is essential to establish a robust regulatory framework, promote public awareness campaigns, and implement responsible gaming measures such as deposit limits, time-out 

In [8]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +-----------+        
      | OllamaLLM |        
      +-----------+        
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

<h1 style="color:rgb(255, 98, 0);">3. Parallel Chain</h1>

### Summary of Flow:

1. Input text is passed into two branches in parallel:

    * One creates notes (prompt1 -> model1 -> parser)

    * One creates quiz (prompt2 -> model2 -> parser)

2. Outputs (notes, quiz) are merged using prompt3 -> model1 -> parser.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

load_dotenv()

model1 = ChatOpenAI()

model2 = ChatAnthropic(model_name='claude-3-7-sonnet-20250219')

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()


```
            +------------------+
            |     Input        |
            +------------------+
                     |
                     v
            +------------------+
            | RunnableParallel |
            +------------------+
               /           \
              /             \
             v               v
+---------------------+   +---------------------+
| prompt1             |   | prompt2             |
| Generate notes      |   | Generate quiz       |
+---------------------+   +---------------------+
             |                       |
             v                       v
+---------------------+   +---------------------+
| model1 (ChatOpenAI) |   | model2 (ChatAnthropic) |
+---------------------+   +---------------------+
             |                       |
             v                       v
+---------------------+   +---------------------+
| parser (StrOutput)  |   | parser (StrOutput)  |
+---------------------+   +---------------------+
             \                       /
              \                     /
               v                   v
           +----------------------------+
           |       Dict Output          |
           |  {'notes': .., 'quiz': ...}|
           +----------------------------+
                     |
                     v
         +-------------------------+
         | prompt3                 |
         | Merge notes and quiz    |
         +-------------------------+
                     |
                     v
         +-------------------------+
         | model1 (ChatOpenAI)     |
         +-------------------------+
                     |
                     v
         +-------------------------+
         | parser (StrOutput)      |
         +-------------------------+
                     |
                     v
                Final Output
```

<h1 style="color:rgb(255, 98, 0);">4. Conditional Chain</h1>

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

model = ChatOpenAI()

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

```
            +------------------+
            |     Input        |
            +------------------+
                     |
                     v
            +---------------------+
            | classifier_chain    |
            | prompt1 -> model    |
            +---------------------+
                     |
                     v
            +---------------------+
            | RunnableBranch      |
            +---------------------+
             /         |          \
            /          |           \
           v           v            v
   [positive]     [negative]      [fallback]
       |               |                |
       v               v                v
 +--------------+ +--------------+ +-----------------+
 | prompt2      | | prompt3      | | RunnableLambda  |
 | Positive     | | Negative     | | "could not find |
 | response     | | response     | | sentiment"      |
 +--------------+ +--------------+ +-----------------+
       |              |                  |
       v              v                  v
 +-------------+ +-------------+ +------------------+
 | model       | | model       | | Final Output     |
 +-------------+ +-------------+ +------------------+
       |             |
       v             v
 +--------------------------+
 | parser (StrOutput)       |
 +--------------------------+
       |
       v
  Final Output (Response)
```
